In [1]:
import re
from typing import Dict, Any, List
import json
import random

In [2]:
filepath = "C:\\projects\\pokefirered-expansion\\src\\data\\trainers.party"

In [3]:
with open(filepath, 'r', encoding="utf8") as file:
    trainers_data = file.read()

In [4]:
def choose_team(pokemon_list: list, average_level: int) -> List[object]:
    team = []

    while len(team) < 6 and pokemon_list:
        poke = random.choice(pokemon_list)
        level = random.randint(average_level - 3, average_level + 3)
        p_data = {
            "name": poke.capitalize(),
            "level": level,
            "ivs": "30 HP / 30 Atk / 30 Def / 30 SpA / 30 SpD / 30 Spe"
        }
        team.append(p_data) 
            
    return team
def should_skip(class_name: str) -> bool:
    skip_class_tokens = ["rs", "boss", "leader", "aqua", "magma", "painter", "rival", "elite four"]
    if any(token in class_name for token in skip_class_tokens):
        return True
    return False
def extract_trainer_data(key: str, text: str) -> Dict[str, Any]:
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    
    trainer_data = {'key': key, 'team': []}
    current_pokemon = None
    
    for line in lines:
        if ':' in line and not line.startswith('-'):
            # Key-value pair; could be trainer metadata or part of a Pokémon
            k, v = line.split(':', 1)
            k = k.strip()
            v = v.strip()
            if current_pokemon is None:
                # Trainer-level metadata
                trainer_data[k.lower().replace(" ", "_")] = v
            else:
                if k.lower() == 'level':
                    current_pokemon['level'] = int(v)
                elif k.lower() == 'ivs':
                    
                    current_pokemon['ivs'] = v
                else:
                    # Arbitrary key-value at Pokémon level
                    current_pokemon[k.lower().replace(" ", "_")] = v
        elif line.startswith('-'):
            # Move line
            if current_pokemon is not None:
                current_pokemon.setdefault('moves', []).append(line[1:].strip())
        else:
            # New Pokémon name (line with no colon or dash)
            if current_pokemon:
                trainer_data['team'].append(current_pokemon)
            current_pokemon = {'name': line.strip()}
    
    if current_pokemon:
        trainer_data['team'].append(current_pokemon)
    
    return trainer_data
def output_trainer_data(trainer: object) -> str:
    t_data = f"=== {trainer['key']} ===\n"
    for key, value in trainer.items():
        if key != "team" and key != "key":
            if key == "ai":
                fixed_key = "AI"
            elif key == "double_battle":
                fixed_key = "Double Battle"
            else:
                fixed_key = key.replace('_', ' ').capitalize()
            t_data += f"{fixed_key}: {value}\n"
        
    t_data += "\n"
    if "team" in trainer:
        for p in trainer["team"]:
            t_data += f"{p['name']}\n"
            t_data += f"Level: {p.get('level', 'N/A')}\n"
            if "ivs" in p:
                t_data += f"IVs: {p['ivs']}\n"
            if "moves" in p:
                m_count = 0
                for m in p["moves"]:
                    if m_count >= 4:
                        break
                    t_data += f"- {m.title()}\n"
                    m_count += 1
                
            t_data += "\n"
    return t_data.strip()

In [5]:
trainers = {}

pattern = r"=== (.*?) ===\n(.*?)(?=\n=== |\n$)"
trainer_blobs = re.findall(pattern, trainers_data, re.DOTALL)

for t in trainer_blobs:
    if t[0] == "TRAINER_BLACK_BELT_KOICHI":
        print(t[1])
    trainers[t[0]] = extract_trainer_data(t[0], t[1])

Name: KOICHI
Class: Black Belt
Pic: Black Belt
Gender: Male
Music: Intense
Double Battle: No
AI: Check Bad Move

Hitmonlee @ Black Belt
Level: 37
IVs: 12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe

Hitmonchan @ Black Belt
Level: 37
IVs: 12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe



In [6]:
trainers["TRAINER_BLACK_BELT_KOICHI"]

{'key': 'TRAINER_BLACK_BELT_KOICHI',
 'team': [{'name': 'Hitmonlee @ Black Belt',
   'level': 37,
   'ivs': '12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe'},
  {'name': 'Hitmonchan @ Black Belt',
   'level': 37,
   'ivs': '12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe'}],
 'name': 'KOICHI',
 'class': 'Black Belt',
 'pic': 'Black Belt',
 'gender': 'Male',
 'music': 'Intense',
 'double_battle': 'No',
 'ai': 'Check Bad Move'}

In [ ]:
# class_pokes = {}
# for k,v in trainers.items():
#     if 'class' in v:
#         c = v["class"].lower()
#         for p in v['team']:
#             if c not in class_pokes:
#                 class_pokes[c] = []
#             poke = p['name'].lower()
#             if poke not in class_pokes[c]:
#                 class_pokes[c].append(poke)

In [ ]:
# delete_classes = []
# for k, v in class_pokes.items():
#     if len(v) == 1 and v[0] == "ekans":
#         delete_classes.append(k)
# for c in delete_classes:
#     del class_pokes[c]

In [ ]:
# # dump the class_pokes dictionary to a JSON file
# output_file = "C:\\projects\\class_pokes.json"
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(class_pokes, f, ensure_ascii=False, indent=4)

In [11]:
output_file = "C:\\projects\\class_pokes.json"
with open(output_file, 'r', encoding='utf-8') as f:
    class_pokes = json.load(f)

In [12]:
for trainer_name in trainers.keys():
    class_name = trainers[trainer_name].get("class", "").lower()
    if should_skip(class_name):
        continue
    if class_name not in class_pokes:
        print(f"Class '{class_name}' not found in class_pokes. Skipping {trainer_name}.")
        continue

    level_sum = 0
    for poke in trainers[trainer_name]["team"]:
        if "level" in poke:
            level_sum += poke["level"]
    avg_level = max(level_sum // len(trainers[trainer_name]["team"]) if trainers[trainer_name]["team"] else 15, 3)
    

    trainers[trainer_name]["team"] = choose_team(class_pokes[class_name], avg_level)
    

Class 'pkmn trainer 1' not found in class_pokes. Skipping TRAINER_NONE.
Class 'interviewer' not found in class_pokes. Skipping TRAINER_INTERVIEWER.
Class 'hex maniac' not found in class_pokes. Skipping TRAINER_HEX_MANIAC.
Class 'rich boy' not found in class_pokes. Skipping TRAINER_RICH_BOY.
Class 'guitarist' not found in class_pokes. Skipping TRAINER_GUITARIST.
Class 'kindler' not found in class_pokes. Skipping TRAINER_KINDLER.
Class 'bug maniac' not found in class_pokes. Skipping TRAINER_BUG_MANIAC.
Class 'school kid' not found in class_pokes. Skipping TRAINER_SCHOOL_KID_M.
Class 'school kid' not found in class_pokes. Skipping TRAINER_SCHOOL_KID_F.
Class 'sr and jr' not found in class_pokes. Skipping TRAINER_SR_AND_JR.
Class 'pokefan' not found in class_pokes. Skipping TRAINER_POKEFAN_M.
Class 'pokefan' not found in class_pokes. Skipping TRAINER_POKEFAN_F.
Class 'expert' not found in class_pokes. Skipping TRAINER_EXPERT_M.
Class 'expert' not found in class_pokes. Skipping TRAINER_EXPE

In [13]:
trainer_parties = ""
for t_name, t in trainers.items():
    trainer_parties += output_trainer_data(t) + "\n\n"

In [14]:
with open(filepath, 'w', encoding='utf-8') as file:
    file.write(trainer_parties)

In [115]:
trainer_parties

'=== TRAINER_NONE ===\nName: PH\nClass: Pkmn Trainer 1\nPic: Hiker\nGender: Male\nMusic: Male\nDouble battle: No\n\n=== TRAINER_AQUA_LEADER ===\nName: PH\nClass: Aqua Leader\nPic: Aqua Leader Archie\nGender: Male\nMusic: Aqua\nDouble battle: No\nAi: Check Bad Move\n\nEkans\nLevel: 5\n0 HP / 0 Atk / 0 Def / 0 SpA / 0 SpD / 0 Spe\n\n=== TRAINER_AQUA_GRUNT_M ===\nName: PH\nClass: Team Aqua\nPic: Aqua Grunt M\nGender: Male\nMusic: Aqua\nDouble battle: No\nAi: Check Bad Move\n\nEkans\nLevel: 5\n0 HP / 0 Atk / 0 Def / 0 SpA / 0 SpD / 0 Spe\n\n=== TRAINER_AQUA_GRUNT_F ===\nName: PH\nClass: Team Aqua\nPic: Aqua Grunt F\nGender: Male\nMusic: Aqua\nDouble battle: No\nAi: Check Bad Move\n\nEkans\nLevel: 5\n0 HP / 0 Atk / 0 Def / 0 SpA / 0 SpD / 0 Spe\n\n=== TRAINER_RS_AROMA_LADY ===\nName: PH\nClass: RS Aroma Lady\nPic: RS Aroma Lady\nGender: Male\nMusic: Female\nDouble battle: No\nAi: Check Bad Move\n\nEkans\nLevel: 5\n0 HP / 0 Atk / 0 Def / 0 SpA / 0 SpD / 0 Spe\n\n=== TRAINER_RS_RUIN_MANIAC ==

In [15]:
trainers["TRAINER_BLACK_BELT_KOICHI"]

{'key': 'TRAINER_BLACK_BELT_KOICHI',
 'team': [{'name': 'Hitmonlee @ Black Belt',
   'level': 37,
   'ivs': '12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe'},
  {'name': 'Hitmonchan @ Black Belt',
   'level': 37,
   'ivs': '12 HP / 12 Atk / 12 Def / 12 SpA / 12 SpD / 12 Spe'}],
 'name': 'KOICHI',
 'class': 'Black Belt',
 'pic': 'Black Belt',
 'gender': 'Male',
 'music': 'Intense',
 'double_battle': 'No',
 'ai': 'Check Bad Move'}